In [1]:
import numpy as np
import scipy.sparse as sparse
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix
from scipy.sparse import csc_matrix
from collections import defaultdict
from sklearn.preprocessing import normalize
import networkx as nx

In [2]:
### DATA AS MATRICES
# S = csr_matrix(np.array([[0, 1, 1, 0],
#                          [1, 0, 0, 1],
#                          [1, 0, 0, 0],
#                          [0, 1, 0, 0]]))

# T = csr_matrix(np.array([[0, 10,10, 0],
#                          [10, 0, 0,10],
#                          [10, 0, 0, 0],
#                          [0, 10, 0, 0]]))

In [3]:
### DATA AS GRAPHS
# STRUCTURE
G=nx.DiGraph()
G.add_edge('a','y')
G.add_edge('y','a')
G.add_edge('a','m')
G.add_edge('m','a')
G.add_edge('y','y')

### TRANSITIONS
H=nx.DiGraph()
H.add_edge('a','y',weight=10)
H.add_edge('a','m',weight=5)
H.add_edge('m','y',weight=1)

### TO UNDIRECTED
G = G.to_directed()
tmp = nx.Graph()
tmp.add_edges_from(H.edges(), weight=0)
for u, v, d in H.edges(data=True):
    tmp[u][v]['weight'] += d['weight']
H = tmp

### ADJACENCY MATRICES
S = nx.adjacency_matrix(G,['y','a','m'])
T = nx.adjacency_matrix(H,['y','a','m'])

In [4]:
##############################################################################
# AIC
##############################################################################
def AIC(loglikelihood, params):
    return (-2 * loglikelihood) + (2 * params)

In [5]:
##############################################################################
# RANDOM WALK
##############################################################################

def random_walk_likelihood(structure, transitions, alpha=0.85):
    params = []
    if alpha is not None and (alpha < 0 or alpha > 1):
        raise ValueError("damping factor must be between 0 and 1 (inclusive).")
        return None, None

    params = [alpha]
    N = structure.shape[0]
    ### rows and columns that matter (where there is a link and transitions)
    nnz = structure.multiply(transitions)
    nnz = sparse.find(nnz)
    ### probability of following sructure: alpha / outlinks
    if alpha > 0:
        P = csr_matrix((alpha * structure) / structure.sum(axis=1))
        ### only cells that matter
        P = P[nnz[0],nnz[1]]
        ### adding teleportation
        P += (1-alpha) / N
    else:
        ### adding teleportation
        P = (1-alpha) * structure[nnz[0],nnz[1]] / N
    ### log-likelihood
    l = (transitions[nnz[0],nnz[1]].A1 * np.log(P.A1)).sum()
    return l,params

In [6]:
##############################################################################
# HOP RANK
##############################################################################

def hop_counts(structure):
    hops = csr_matrix(structure.shape, dtype=np.int16)
    previous_hops = None
    
    hop = 1
    structure.setdiag(0)
    m = structure.copy()

    for r,row in enumerate(structure):
        hops[r,row.indices] = hop
        
    while m.sum() > 0:
        
        if previous_hops is None:
            previous_hops = m.copy()
        else:
            previous_hops += m
    
        m = lil_matrix(m.dot(structure))
        m.setdiag(0)
        m -= previous_hops
        m = (m>0).astype(np.int8)
        m.eliminate_zeros()
        
        hop += 1
        for r,row in enumerate(m):
            hops[r,row.indices] = hop

    hops.setdiag(0) 
    return hops

def hop_alphas(transitions, hops):    
    maxhops = int(hops.max())
    total = transitions.sum()
    alphas = np.zeros(maxhops)
    counts = np.zeros(maxhops)
    
    for hop in range(maxhops):
        transitions_in_hop = np.any(hops==hop+1).astype(np.int8).multiply(transitions)
        counts[hop] = transitions_in_hop.sum()
        alphas[hop] = transitions_in_hop.sum() / total
        
    return alphas, counts

def hop_rank_likelihood(structure, transitions):
    hops = hop_counts(structure)
    alphas, counts = hop_alphas(transitions, hops)
    l = 1.0
    for r, row in enumerate(transitions):
        if row.sum() > 0:
            lrow = csr_matrix(np.take(alphas,hops[r].data-1)[row.indices],dtype=np.float)
            l += (row.data * np.log(lrow.data)).sum()
    return l,alphas

In [7]:
##############################################################################
# MARKOV CHAIN
##############################################################################

def markov_chain_likelihood(structure, transitions):
    P = normalize(transitions,'l1',axis=1)
    l = (transitions.data * np.log(P.data)).sum()
    params = sparse.find(P.tocsr())[2].tolist()
    return l,params

In [8]:

### LOG-LIKELIHOODS
alpha = 0.
l,params = random_walk_likelihood(S,T,alpha)
aic = AIC(l,len(params))
print('random walk {}: \n- likelihood: {}\n- AIC: {}\n'.format(params,l,aic))

alpha = 1.
l,params = random_walk_likelihood(S,T,alpha)
aic = AIC(l,len(params))
print('random walk {}: \n- likelihood: {}\n- AIC: {}\n'.format(params,l,aic))

alpha = 0.85
l,params = random_walk_likelihood(S,T,alpha)
aic = AIC(l,len(params))
print('random walk {}: \n- likelihood: {}\n- AIC: {}\n'.format(params,l,aic))

l,params = hop_rank_likelihood(S,T)
aic = AIC(l,len(params))
print('hop rank {}: \n- likelihood: {}\n- AIC: {}\n'.format(params,l,aic))

l,params = markov_chain_likelihood(S,T)
aic = AIC(l,len(params))
print('markov chain {}: \n- likelihood: {}\n- AIC: {}\n'.format(params,l,aic))


random walk [0.0]: 
- likelihood: -32.9583686600433
- AIC: 67.9167373200866

random walk [1.0]: 
- likelihood: -17.328679513998633
- AIC: 36.657359027997266

random walk [0.85]: 
- likelihood: -19.13781445197653
- AIC: 40.27562890395306

hop rank [ 0.9375  0.0625]: 
- likelihood: -6.481333078606697
- AIC: 16.962666157213395

markov chain [0.6666666666666666, 0.16666666666666666, 0.9090909090909091, 0.8333333333333334, 0.09090909090909091, 0.3333333333333333]: 
- likelihood: -15.60207684846164
- AIC: 43.204153696923285



/usr/local/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
